<a href="https://colab.research.google.com/github/DaryaTereshchenko/HateSpeechDetection/blob/main/RoBerta_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install transformers

In [ ]:
!pip install tensorflow_text

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from transformers import RobertaTokenizer, RobertaConfig, TFRobertaPreTrainedModel
from transformers import TFRobertaMainLayer
import tensorflow as tf

In [ ]:
ethos_hate = pd.read_csv("ethos_hate.csv", sep=",", encoding="unicode_escape").drop(columns="label")
ethos_hate["label"] = 1

ethos_neutral = pd.read_csv("ethos_neutral.csv", sep=",", encoding="unicode_escape").drop(columns="label")
ethos_neutral["label"] = 0

train_n = ethos_neutral.sample(frac=0.9, random_state=0)
test_n = ethos_neutral.drop(train_n.index)

train_hate = ethos_hate.sample(frac=0.9, random_state=0)
test_hate = ethos_hate.drop(train_hate.index)

train = pd.concat([train_n, train_hate]).sample(frac=1, random_state=300).reset_index(drop=True)
test = pd.concat([test_n, test_hate]).sample(frac=1, random_state=300).reset_index(drop=True)

print(train.text[0])
print(test.shape)

What episode number was this?
(100, 2)


In [ ]:
def clean_text(line):
  stops = set(stopwords.words("english"))
  processed_line = line.lower()
  words = [word for word in word_tokenize(processed_line)]
  processed_line = [word for word in words if word not in stops 
                    and word not in set(string.punctuation)]
  text = " ".join(processed_line)
  return text

In [ ]:
# Sentences for embeddings
train_sentence_list = list(map(clean_text, train.text.values))
test_sentence_list = list(map(clean_text, test.text.values))

In [ ]:
train_sentence_list

In [ ]:
# df_train = pd.DataFrame(list(map(lambda i: np.asarray(i), train_sentence_list)))
# df_test = pd.DataFrame(list(map(lambda i: np.asarray(i), test_sentence_list)))

In [ ]:
MODEL_NAME = 'roberta-base'
MAX_LEN = 128
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
def to_tokens(input_text, tokenizer):
    output = tokenizer.encode_plus(input_text, max_length=MAX_LEN, padding="max_length")
    return output

def select_field(features, field):
    return [feature[field] for feature in features]

In [99]:
tokenizer_output_train = [to_tokens(sent, tokenizer)  for sent in train_sentence_list]
tokenizer_output_test = [to_tokens(sent, tokenizer)  for sent in test_sentence_list]

We create and store the input ids as well as attention masks as an numpy array. The input ids are the numbers which are understood by the Neural Network.

In [104]:
input_ids_train = np.array(select_field(tokenizer_output_train, 'input_ids'), dtype=object)
attention_masks_train = np.array(select_field(tokenizer_output_train, 'attention_mask'), dtype=object)

In [108]:
input_ids_test = np.array(select_field(tokenizer_output_test, 'input_ids'), dtype=object)
attention_masks_test = np.array(select_field(tokenizer_output_test, 'attention_mask'), dtype=object)